In [1]:
#IPython extension to reload modules before executing user code. If we make any change in function 
# and imoprting it we don't need to restart if we use auto reload 
%load_ext autoreload

# # to print in between values as well without print
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"


#hide warnings
import warnings
warnings.filterwarnings('ignore')

In [82]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pickle

# sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [83]:
# to save pickle file
def save_pkl_model(path,model):
    pickle.dump(model, open(path,'wb'))

In [84]:
# to load
def load_pkl_model(path):
    sc = pickle.load(open(path,'rb'))
    return sc

In [85]:
def preprocess_pipeline_1(test):
    miss_num_imputer = load_pkl_model(missing_num_imputer_path)
    missing_cat_imputer = load_pkl_model(missing_cat_imputer_path)
    ohe_converter = load_pkl_model(ohe_converter_path)
    bx_converter = load_pkl_model(bx_converter_path)

    # processing numeric
    num_cols = test.select_dtypes(include='number')
    num_column_names = num_cols.columns
    num_miss_processed = miss_num_imputer.transform(num_cols)
    num_scalled = bx_converter.transform(num_miss_processed)
    num_scalled_df = pd.DataFrame(num_scalled,columns=num_column_names)


    # processing categoric
    cat_cols = test.select_dtypes(include='O')
    cat_miss_processed = missing_cat_imputer.transform(cat_cols)
    cat_encoded = ohe_converter.transform(cat_miss_processed).toarray()
    cat_encoded_df = pd.DataFrame(cat_encoded,columns=ohe_converter.get_feature_names_out())


    # combine categoric and numeric data and return
    test_processed = pd.concat([num_scalled_df,cat_encoded_df],axis=1)
    
    return test_processed

In [86]:
# spacify saved preprocessing model paths
missing_num_imputer_path = 'Data/processing_models/imputer_num.pkl'
missing_cat_imputer_path = 'Data/processing_models/imputer_cat.pkl'
ohe_converter_path = 'Data/processing_models/ohe_converter.pkl'
bx_converter_path = 'Data/processing_models/ss_converter.pkl'

In [87]:
train_data = pd.read_csv('Data/transformed/train_scale_bxtransform_ohe.csv')
train_data.head()

,raisedhands,VisITedResources,AnnouncementsView,Discussion,gender_M,NationalITy_Iran,NationalITy_Iraq,NationalITy_Jordan,NationalITy_KW,NationalITy_Lybia,...,Topic_Math,Topic_Quran,Topic_Science,Topic_Spanish,Semester_S,Relation_Mum,ParentAnsweringSurvey_Yes,ParentschoolSatisfaction_Good,StudentAbsenceDays_Under-7,Class
0,0.741477,0.413580,0.301585,0.013122,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,M
1,-1.125797,-0.764168,-1.418531,-1.009282,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,L
2,0.065395,0.443779,-0.708048,1.108555,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,H
3,0.805865,0.957156,1.273825,1.619757,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,M
4,0.741477,1.108150,0.451161,-1.374426,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,H


In [133]:
# load test data and check
test_data = pd.read_csv('Data/test.csv')
# test_data.head()
# devide Xtest ytest to compare
X_test,y_test_org = test_data.loc[:,~test_data.columns.isin(['Class'])],test_data['Class']
# preporcess the data for the model input
X_test_processed = preprocess_pipeline_1(X_test)
# X_test_processed.head()

In [134]:
# label_to_key = {}
# key_to_label = {}
# for i,label in enumerate(train_data['Class'].unique()):
#     label_to_key[label] = i
#     key_to_label[i]= label

In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD


In [136]:
def ohe_converter(train):
    ohe = OneHotEncoder(handle_unknown = 'ignore') 
    encoded_vec = ohe.fit(train)
    ohe_transformed = encoded_vec.transform(train).toarray()
    return ohe_transformed,ohe

In [141]:
X_train = train_data.iloc[:,:-1].values
# y_train = train_data.iloc[:,-1].apply(lambda x: label_to_key[x]).values
# y_train = train_data.iloc[:,-1].values
y_train = train_data.iloc[:,-1].values
y_train = y_train.reshape(-1,1)
y_train,ohe_train = ohe_converter(y_train)

X_test = X_test_processed.values
# y_test = y_test.apply(lambda x: label_to_key[x]).values
y_test = y_test_org.values
y_test = y_test.reshape(-1,1)
y_test = ohe_train.transform(y_test).toarray()

In [142]:
X_train.shape

(384, 60)

In [143]:
X_test.shape

(96, 60)

In [144]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)

In [145]:
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

In [146]:
# Pytorch train and test sets
train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

In [147]:
# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = 5, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = 5, shuffle = False)

In [148]:
# next(iter(train_loader))

In [149]:
# Display image and label.
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([5, 60])
Labels batch shape: torch.Size([5, 3])


In [150]:
# train_features.view(1,5,60).shape

In [151]:
# Create ANN Model
# we create a new class ANNModel which inherits from nn.Module 
# define variables in a class and a forward function which uses those variables and perform forward pass 
class ANNModel(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ANNModel, self).__init__()
        
        # Linear function 1: 60 --> 32
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        # self.relu1 = nn.ReLU()
        
        # Linear function 2: 32 --> 32
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        # self.tanh2 = nn.Tanh()
        
        # Linear function 3: 32 --> 32
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        # self.elu3 = nn.ELU()
        
        # Linear function 4 (readout): 32 --> 1
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function 1
        out = F.relu(self.fc1(x))
        # Non-linearity 1
        # out = self.relu1(out)
        
        # Linear function 2
        out = F.relu(self.fc2(out)) 
        # Non-linearity 2
        # out = self.relu1(out)
        
        # Linear function 2
        out = F.relu(self.fc3(out))
        # Non-linearity 2
        # out = self.relu1(out)
        
        # Linear function 4 (readout)
        out = self.fc4(out)
        return out
        # return F.softmax(out,dim=1)

In [152]:
# instantiate ANN
input_dim = train_features.size()[1]
hidden_dim = 32 #hidden layer dim is one of the hyper parameter and it should be chosen and tuned. For now I only say 150 there is no reason.
output_dim = 3

# Create ANN
model = ANNModel(input_dim, hidden_dim, output_dim)
# to check all working fine, we can simply call model, no need to call specificly forward
# model(train_features)
# model.forward(train_features)

In [153]:
# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# SGD Optimizer
# model parameters are those variables for which gradient calculation is true, which can be updated
learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [154]:
model.parameters

<bound method Module.parameters of ANNModel(
  (fc1): Linear(in_features=60, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=3, bias=True)
)>

In [155]:
EPOCHS = 50

for epoch in range(EPOCHS):
    for data in train_loader:
        # get the batch
        X,y = data
        # make the gradients zero so get ready for next pass else keep getting add
        model.zero_grad()
        output = model(X)

        # depends if our data is one hot encoded 
        # print(output)
        # print(y)
        loss =error(output,y)

        # calculate gradient
        loss.backward()

        # update gradient
        optimizer.step()
        print(loss)        

tensor(1.1426, grad_fn=<DivBackward1>)
tensor(1.1742, grad_fn=<DivBackward1>)
tensor(1.1444, grad_fn=<DivBackward1>)
tensor(1.1645, grad_fn=<DivBackward1>)
tensor(1.1270, grad_fn=<DivBackward1>)
tensor(1.0573, grad_fn=<DivBackward1>)
tensor(1.1138, grad_fn=<DivBackward1>)
tensor(1.1070, grad_fn=<DivBackward1>)
tensor(1.1455, grad_fn=<DivBackward1>)
tensor(1.1036, grad_fn=<DivBackward1>)
tensor(1.1838, grad_fn=<DivBackward1>)
tensor(1.1489, grad_fn=<DivBackward1>)
tensor(1.1226, grad_fn=<DivBackward1>)
tensor(1.0706, grad_fn=<DivBackward1>)
tensor(1.1170, grad_fn=<DivBackward1>)
tensor(1.0696, grad_fn=<DivBackward1>)
tensor(1.0842, grad_fn=<DivBackward1>)
tensor(1.0827, grad_fn=<DivBackward1>)
tensor(1.1468, grad_fn=<DivBackward1>)
tensor(1.1849, grad_fn=<DivBackward1>)
tensor(1.1168, grad_fn=<DivBackward1>)
tensor(1.1433, grad_fn=<DivBackward1>)
tensor(1.1485, grad_fn=<DivBackward1>)
tensor(1.1371, grad_fn=<DivBackward1>)
tensor(1.1053, grad_fn=<DivBackward1>)
tensor(1.1509, grad_fn=<D

In [156]:
correct = 0
total = 0
results = []

with torch.no_grad():
    for t_data in test_loader:
        X_t,y_t = t_data
        test_output = model(X_t)
        for idx, i in enumerate(test_output):
            res = torch.argmax(i)
            ohe_res = np.zeros(shape = (1,3))[0]
            ohe_res[res] = 1
            labeled_out = ohe_train.inverse_transform(ohe_res.reshape(1,-1))
            results.append(labeled_out[0][0])    

In [121]:
list(results)

['H',
 'H',
 'M',
 'H',
 'L',
 'M',
 'L',
 'L',
 'M',
 'H',
 'M',
 'L',
 'H',
 'H',
 'L',
 'M',
 'H',
 'H',
 'H',
 'L',
 'H',
 'L',
 'H',
 'H',
 'M',
 'L',
 'H',
 'M',
 'H',
 'H',
 'L',
 'M',
 'L',
 'M',
 'H',
 'H',
 'H',
 'H',
 'H',
 'M',
 'L',
 'L',
 'H',
 'L',
 'H',
 'M',
 'M',
 'L',
 'H',
 'H',
 'H',
 'H',
 'H',
 'M',
 'M',
 'L',
 'L',
 'M',
 'L',
 'M',
 'L',
 'M',
 'M',
 'H',
 'L',
 'H',
 'H',
 'H',
 'H',
 'L',
 'H',
 'M',
 'H',
 'H',
 'M',
 'M',
 'H',
 'H',
 'H',
 'L',
 'M',
 'L',
 'H',
 'M',
 'L',
 'M',
 'L',
 'L',
 'M',
 'H',
 'L',
 'L',
 'M',
 'M',
 'M',
 'L']

In [158]:
list(y_test_org)

['H',
 'M',
 'M',
 'M',
 'L',
 'M',
 'M',
 'L',
 'M',
 'M',
 'M',
 'L',
 'H',
 'H',
 'L',
 'M',
 'H',
 'M',
 'M',
 'L',
 'M',
 'L',
 'M',
 'M',
 'M',
 'L',
 'H',
 'M',
 'H',
 'H',
 'M',
 'M',
 'L',
 'M',
 'H',
 'H',
 'H',
 'H',
 'H',
 'L',
 'L',
 'L',
 'M',
 'L',
 'H',
 'M',
 'M',
 'L',
 'H',
 'H',
 'H',
 'H',
 'H',
 'M',
 'H',
 'L',
 'L',
 'M',
 'L',
 'M',
 'L',
 'M',
 'M',
 'H',
 'L',
 'M',
 'H',
 'H',
 'H',
 'L',
 'H',
 'M',
 'H',
 'H',
 'M',
 'M',
 'H',
 'M',
 'H',
 'M',
 'M',
 'L',
 'H',
 'L',
 'L',
 'M',
 'M',
 'L',
 'M',
 'M',
 'L',
 'L',
 'M',
 'M',
 'M',
 'M']

In [159]:
# sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import classification_report

In [160]:
report = classification_report(list(y_test_org),results,output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
H,0.674419,1.000000,0.805556,29.000000
L,0.766667,0.920000,0.836364,25.000000
M,0.913043,0.500000,0.646154,42.000000
accuracy,0.760417,0.760417,0.760417,0.760417
macro avg,0.784710,0.806667,0.762691,96.000000
weighted avg,0.802840,0.760417,0.743840,96.000000
